# How to use regioinvent

To be able to use regioinvent, we will need:
- to install the brightway2 Python library (brightway2 and NOT brightway2.5)
- a brightway project within which there is an ecoinvent database with the version 3.9.1 cut-off (only supported version so far)
- to download the trade database stored here: https://doi.org/10.5281/zenodo.11583815 (was extracted from the UN COMTRADE database)

In [1]:
import pandas as pd
import numpy as np
import sys
# change the path here to wherever you stored the Regioinvent Python package
sys.path.append('C://Users/11max/PycharmProjects/Regioinvent/')
import regioinvent

The initialization of the Regioinvent class requires 4 arguments:
- the path to the trade database
- the name you want to to give to the resulting regionalized ecoinvent database created
- the name of your brightway2 project
- the name of the ecoinvent v3.9.1 cut-off database in your brightway2 project

In [2]:
regio = regioinvent.Regioinvent(trade_database_path='C://Users/11max/PycharmProjects/Regioinvent/trade_data.db', 
                                regioinvent_database_name='Regioinvent', 
                                bw_project_name='ecoinvent3.9', 
                                ecoinvent_database_name='ecoinvent3.9.1 cut-off')

2024-06-11 12:59:16,251 - Regioinvent - INFO - Extracting ecoinvent to wurst...


Getting activity data


100%|██████████| 21238/21238 [00:00<00:00, 129903.55it/s]


Adding exchange data to activities


100%|██████████| 674593/674593 [00:44<00:00, 15034.32it/s]


Filling out exchange data


100%|██████████| 21238/21238 [00:03<00:00, 6567.85it/s] 


The initialization first uses the wurst Python library to extract your ecoinvent database from brightway into a list of dictionnary format. This will tremendously speed up modifications made to the database.

Next, we extract and format the export data from the trade database

In [3]:
regio.format_export_data()

2024-06-11 13:00:10,268 - Regioinvent - INFO - Extracting and formatting export data from UN COMTRADE...


We estimate the domestic production data based on the ratio of domestic vs export of the EXIOBASEv3.8.2 database 2019. This ratio is applied to self.export_data.

In [4]:
regio.estimate_domestic_production()

2024-06-11 13:01:34,626 - Regioinvent - INFO - Estimating domestic production data...


We apply first order regionalization, that is for the 1870 traded products of the ecoinvent3.9.1 cut-off database, we determine the exporters of each product (through self.export_data) and then regionalize the following key inputs:
- electricity
- municipal solid waste
- heat

In [5]:
regio.first_order_regionalization()

2024-06-11 13:01:35,723 - Regioinvent - INFO - Regionalizing main inputs of traded products of ecoinvent...
100%|██████████| 1870/1870 [18:36<00:00,  1.67it/s] 


We then extract and format the import data from the trade database

In [6]:
regio.format_import_data()

2024-06-11 13:20:12,718 - Regioinvent - INFO - Extracting and formatting import data from UN COMTRADE...


With the domestic production and import data in hand, we can determine consumption markets for each of the 1870 products.

In [7]:
regio.create_consumption_markets()

2024-06-11 13:21:41,929 - Regioinvent - INFO - Creating consumption markets...
100%|██████████| 1870/1870 [42:54<00:00,  1.38s/it] 


We finally relink created consumption markets to regionalized processes across the regioinvent database.

In [8]:
regio.second_order_regionalization()

2024-06-11 14:12:48,673 - Regioinvent - INFO - Performing second order regionalization...
2024-06-11 14:13:04,852 - Regioinvent - INFO - Finished, you can now write the database...


All regionalized processes are now in a Python dictionnary. We can now write this database to the brightway2 software to be able to use brightway2 or activity-browser to perform LCAs.

In [ ]:
regio.write_to_database()